In [22]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as fun
import pandas as pd
from data import get_data
(x_train, y_train, x_test, y_test) = get_data()


In [39]:

def df_to_tensor(df: pd.DataFrame):
    return torch.from_numpy(df.values).float()

# convert classes to numerical data
def to_numerical(df: pd.Series, positive_class = 'g'):
    new = df.copy()
    for i, v in enumerate(new.values):
        if v == positive_class:
            new.iloc[i] = 1.
        else:
            new.iloc[i] = 0.
    return new.astype(float)

x_train = df_to_tensor(x_train)
x_test = df_to_tensor(x_test)
y_train = df_to_tensor(to_numerical(y_train))
y_test = df_to_tensor(to_numerical(y_test))

In [40]:
# loaders
BATCH_SIZE = 64

trainset = torch.utils.data.DataLoader((x_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
testset = torch.utils.data.DataLoader((x_test, x_test), batch_size=BATCH_SIZE, shuffle=True)

In [44]:
# model
class Model(nn.Module):
    def __init__(self, l1_nodes, l2_nodes, input_size = 10):
        super().__init__()
        self.l1 = nn.Linear(input_size, l1_nodes)
        self.l2 = nn.Linear(l1_nodes, l2_nodes)
        self.l1 = nn.Linear(l2_nodes, 1)

    def forward(self, x):
        x = fun.relu(self.l1(x))
        x = fun.relu(self.l2(x))

        x = fun.sigmoid(self.l1(x)) # TODO: sigmoid or no?
        return x

L1_NODES = 20
L2_NODES = 20

model = Model(L1_NODES,L2_NODES)